# 筋電の周波数解析例

学籍番号：  
氏　　名：

## 0. 準備

□0-1. pandas，NumPy，matplotlibをインポートする。さらに線形回帰分析（相関分析）を行いたいので，`scipy.stats`を`stat`でインポートしておく。

□0-2. データフレーム`df`に`'kinden-data.csv'`を読み込む：  
> `'kinden-data.csv'`はセンサのデータCSVファイルから筋電のみを取り出し，CSVファイルに書き出したファイルである。

□0-3. サンプリング周波数を`H`（=1000[Hz]）で定義しておく：

## 1. 筋電データ配列の作成

□1-1. dfにおいて，`'t'`は実験時刻[ms]，`'bi'`が二頭筋，`'tri'`が三頭筋の表面筋電圧（筋電とする）[mV]である。それぞれの筋電の1D-arrayを`bi`，`tri`としておく：

□1-2. `bi`と`tri`をプロットする。

□1-3. 前方index0〜50000までをプロットしてみる。

> index20000辺りからは確実に安定している。

□1-4. 後方index300000〜をプロットしてみる。

> index330000程度まで安定している。

□1-5. したがって，`bi`, `tri`をindex20000~330000まで切り取り，それらを再定義しておく：

□1-6. 再定義した `bi`, `tri `をプロットしてみる。

> 良さそうだ。これらの（再定義した）bi, triのndarrayでやっていこう。

□1-7. `bi`と`tri`の要素数（一致する）を確認しよう：

□1-8. 時間を区切って一定時間間隔のデータブロックをつくり，スペクトルの時間的変化をみていきたい。そうしてFFTを行うのだが，FFTは$2^n$個のデータに対して行うのだったし，このときのデータ数はサンプリング周波数の1000[Hz]以上でなければならなかった。  

そこで，とりあえず，$2^n$の値を各$n$について調べてみる(n=0,1,2,...20)：

□1-9. $2^{10}=1024$であることより，$n$は$10$以上でなければならない（サンプリング周波数が1000なので）。さて，**$n$が大きければ大きいほどFFTの周波数分解能は高くなるが，その代わり時間分解能は低くなる**ことに注意しよう（時間間隔のブロックの数が少なくなるからだ）。ということで，周波数分解能と時間分解能を天秤にかけて$n$を決めるんだ。  

そこで，$n$を10から増加させていったときのデータブロックの数を計算してみよう（データ要素数は`len(bi)`または`len(tri)`であることに注意）：

> この結果から，自分たちの実験結果の解析のときの$n$の値を決定せよ。

□1-10. $n=12$で解析してみることにしよう。したがって，その時の各ブロックのデータ数`N`は4096個である。

□1-11. １つの間隔について`N`(=4096)データ，間隔数`len(bi)//N`(=75)だから，ndarray `bi`と`tri`の要素数を，間隔がきっちり収まるように再定義しておく（端数になる要素を切り捨てる）：

□1-12. `bi`または`tri`の要素数を確かめる：

> 要素数は30720まで減った。

□1-13. 再再定義した bi, triをプロットしてみる。

□1-14. `bi`, `tri`を`len(bi)//N`(=75)のブロックにわけ，それぞれ`lisbi`，`listri`に格納する。  
> **【公式1】**  
> 1D-ndarray `arr`を等間隔のブロックにわけるには：
> ```
> np.array_split(arr, ブロック数)
> ```
> を使う。戻り値はリストで，各リストの要素が分割された1D-ndarrayである（ <a href="https://note.nkmk.me/python-numpy-split/">NumPy配列ndarrayを分割（split, array_split, hsplit, vsplit, dsplit）</a> 参照）。  
> ※）`arr`の要素数がブロック数で割り切れない場合はエラーを出す。

> エラーが出なかったらちゃんと分割されている。

□1-15. リスト `lisbi`の要素数（分割されたブロック数）を`blocks`とし，その数を確認しておく。

> サブセクション1-9のとおり，75個の要素（１つの要素は1D-ndarrayである）がある。

## 2. 有効なパワーの1D-ndarrayの作成

□2-1. ここからは二頭筋のみで解説する。とりあえず1D-ndarray `lisbi`を10要素分表示してみる：

> 1D-ndarrayを要素にもつリストで，各要素の1D-ndarrayが筋電データで，それらが時間的に連続して格納されている。

□2-2. FFT周波数を`arrf`を求めておく：  
> index $k$のFFT周波数が$f_k=\frac{kH}{N}$であることを思い出せ。

□2-3. ナイキスト周波数は当然500(=H/2)[Hz]だが，1D-ndarray `arrf`のちょうど真ん中にある要素を取り出して確認してみる：

> 500だ。もしそうでないなら，サブセクション2-2で実行したことが間違っている。

□2-4. 有効なFFT周波数は0Hzからナイキスト周波数未満の周波数であるから，この有効周波数の1D-ndarrayをarrf_effという変数名に格納しておこう：

□2-5. いよいよ二頭筋の筋電をFFTしてみる。リスト`lisbi`は，`N`(=4098)の要素をもった筋電配列(1D-array)が`blocks`(=75)個並んでいるリストであった。したがって，for文で`lisbi`から１つずつ1D-ndarrayの要素をとってきてFFT関数を作用させなければならないように思える。しかしながら，NumPyのFFT関数はユニバーサル関数になっている。  

**ユニバーサル関数**とは**リストやndarrayの全要素に対して，要素ごとに演算等の処理を行い，結果をndarrayで返す関数**である。したがって，`np.fft.fft`は`lisbi`に作用させたとき，`lisbi`の各々の要素の1D-ndarrayをFFT処理し，それらの結果を１つのndarrayでまとめて返すのである。  

`lis_bi`の要素をまとめてFFT処理した結果のndarrayを`arrF`に格納する：  
> ただし，この授業でのFFTの規約上，すべてのFFTの結果を`N`(=2$^{12}$=4098)で除しておくが，ここでの解析にはこのことは結果になにも影響を及ぼさないので，レポートにおける分析には`N`で除しておく必要はない。  
※）`/`演算（除演算）もユニバーサルである。

□2-6. `arrF`を表示してみる：

> すばらしい。ユニバーサル関数は強力だ。

□2-7. `arrF`のパワー`arrP`も求めておく：  
> ※）`np.abs`関数および`**`演算（累乗演算）もユニバーサルである。

□2-8. まずは全てのパワー（`arrP`の全ての要素）をプロットし，パワーがどのような形状になるかを確かめてみる。ただし，周波数0Hzのときのパワー（非常におおきな値を示すであろうが，それは装置に依存するバイアス値であろうから解析に無意味）と，ナイキスト周波数以上のパワーを除去しておく：

□2-9. 低周波数（例えば0~5Hzとしておこう）でのパワーは，多分，皮膚の伸長・短縮による電極間距離の振動など，力学的な要因によって引き起こされるの電圧変化からくるものと考えられる（0Hzはバイアス）。他方，高周波数（例えば300Hz以上としておこう）でのパワーは，デバイスのノイズによって引き起こされるものと考えられる。したがって，5Hz以上300Hz以下のパワーを有効パワーとして，それ以外の周波数のパワーを除去したい。  

そこでまず，$f_k=\frac{kH}{N} \rightarrow k=\frac{f_k N}{H}$の関係より，5Hzと300Hz近傍のインデックス（$k$）を求め，それらをそれぞれ`k5`, `k300`という変数に格納しておき，それらを表示しよう：

□2-10. セクション2-9のように，有効なパワーは`k2`(=20)インデックス（の周波数）〜`k300`(=1228)インデックス（に対応する周波数）までである。そこで，パワーのセット`arrP`から有効なパワーのセット`arrP_eff`を次のように作成しよう：  
1. `arrP`のナイキスト成分までのコピーを`arrP_eff`につくる。  
> `arrP_eff = arrP[:, :N//2].copy()`とする。こうしなければ，`arrP_eff`を変更させたときに`arrP`も変更されてしまう（<a href="https://note.nkmk.me/python-numpy-view-copy-shares-memory/">NumPy配列ndarrayがビューかコピーか、メモリを共有しているか判定</a> 参照）。
2. `arrP_eff`の各要素（各パワーの1D-ndarray）の最初〜`k5`インデックスまでの要素を0にする。
3. `arrP_eff`の各要素（各パワーの1D-ndarray）の`k300`インデックス〜最後までの要素を0にする。

□2-12. まずは全ての有効パワー（`arrP/eff`の全ての要素）をプロットしてみる：

> これでいいだろう。

## 3. 平均パワーの時間的変化

□3-1. 各実験時間ブロックの有効パワーの平均値を`arrP_mean`に格納しよう（教科書p.131参照）：

□3-2. １実験時間ブロックは`N`(=4098)[ms]で，セクション1-15より，これが`blocks`(=75)個連なる。そこで各時間ブロックを代表する時刻を各々の**中間時刻**で定め，それを`arrt_mid`という変数に定義しておく：
> 0から始まる`blocks`(=75)個の1ずつ増加する1D-ndarrayをつくり，それに`N/H`(=4.096)[s]を乗じたものに，`N/(2*H)`(=2.048)[s]を加算しておけばよい。

□3-3. `arrt_mid`と`arrP_mean`の散布図を描いてみる：

> 上のグラフをじーっと見ていると，指数関数的に増加しているような気がする。

□3-4. そこで，`arrP_mean`の対数をとって，それを`arrLogP_mean`という変数に格納する。つまり，$\bar{P}_\mathrm{eff}=e^{at+b}$なら$\log{\bar{P}_\mathrm{eff}}=at+b$という１次式で書けるということを利用する。ここで，$\bar{P}_\mathrm{eff}$は平均パワー，$t$は時間（中間時刻），$a$と$b$は定数である。  
> **【公式1】**1D-ndarray `arr`に対して，
> ```
> np.log(arr)
> ```
> は，`arr`の自然対数を返す。`np.log`関数はユニバーサルである。

□3-5.  `arrt_mid`と`arrP_mean`の散布図を描いてみる：

□3-4. 上の散布図にフィットする直線の方程式がどうなるのかを求めてみる。これは$y=ax+b$なる１次方程式の係数$a,b$を，**最小二乗法**$^*$というやり方で求めるやりかただ。  
> $^*$簡単にいうと，データ`arr1`(=$x$)と`arr2`(=$y$)があり，それらの間に$y=ax+b$という関係の予測曲線(直線)をあてはめたとき，**曲線からデータの外れの２乗を最小にする**ように，１次方程式の未定係数$a$，$b$を定める方法である。

ここで今の場合，$x$には1D-ndarray`arrt_mid`が，$y$には1D-ndarray`arrLogP_mean`が入る。NumPyではこれを求める関数がある。 
> ***
> **【公式2】**  1D-ndarrayデータ`arr1`を$x$，1D-ndarrayデータ`arr2`を$y$とするとき，
> ```
> np.polyfit(arr1, arr2, 1)
>```
> は１次関数$y=ax+b$の係数$a$，$b$を1D-ndarrayで返す。   
>***

早速やってみよう：  
> 1. `np.polyfit`関数を用いて，変数$x$に対応にする$t$(=`arrt_mid`)と変数$y$に対応する$\log{\bar{P}_\mathrm{eff}}$(=`arrLogP_mean`)が１次関数$y=a_1x+b_1$にフィットするように係数$a_1$(=`a1`)，$b_1$(=`b1`)を決定する。
> 2. 1D-ndarray `arrt_mid`を独立変数$x$に対応するデータとし，上記1で決定された係数`a1`，`b1`を用いて従属変数$y$に対応する1D-ndarray `arrLogP_fit`をつくる。
> 3. `a1`，`b1`を
> ```
> a1=..., b1=...
> ```
>という形式で表示する。

□3-5. 上で求めた直線を，セクション3-3の散布図の中に描いてみよう：
> - 図のサイズは`(8, 6)`
> - `arrt_mid`と`arrLogP_mean`の間の関係は青い点
> - `arrt_mid`と`arrLogP_fit`の間の関係は赤の曲線
> - x軸ラベルは`'t[sec]'`
> - y軸ラベルは`'log of mean of Power'`
> - 凡例は`['data', 'fitting line']`
> - グリッド線を引く

> ばっちり直線に沿っている。これで，相関係数が求められる。

□3-6. $t$(=`arrt_mid`)と$\log{\bar{P}_\mathrm{eff}}$(=`arrLogP_mean`)に相関があるかどうかを調べよう。  
> **【公式3】** データ1D-ndarray `arr1`と`arr2`の間に相関があるかどうかを調べるには，
```
stat.pearsonr(arr1, arr2)
```
> とする。戻り値はタプルで与えられ，それのインデックス0の要素は**相関係数**を与え，インデックス1の要素は$p$値を与える。$p$値が0.05未満のときのみ`arr1`と`arr2`相関が認められ（相関係数が無意味でない），相関係数による相関の目安は表１のようになる。

<center>表１．相関係数と相関の目安</center>

|相関係数の値|目安|
|:---:|:---:|
|[0.7, 1]|強い正の相関|
|[0.4, 0.7)|正の相関|
|[0.2, 0.4)|弱い正の相関|
|(-0.2, 0.2)|相関なし|
|(-0.4, -0.2]|弱い負の相関|
|(-0.7, -0.4]|負の相関|
|[-1.0, -0.7]|強い負の相関|

> なお，$p$値は，次の仮説が起こりうる確率を算出する：
>
> 相関関係の**帰無仮説**…データ`arr1`とデータ`arr2`の間に相関はない。

> $p<0.05$なので相関が認められ，相関係数から$t$(=`arrt_mid`)と$\log{\bar{P}_\mathrm{eff}}$(=`arrLogP_mean`)の間に強い正の相関が認められる。これは，**$t$(=`arrt_mid`)と$\bar{P}_\mathrm{eff}$(=`arrP_mean`)の間にも強い正の相関が認められる**ことを示している。

## 4. 平均パワー周波数の時間的変化

**平均パワー周波数（mean power frequency：MPF）$f_\mathrm{MP}$**は，有効なパワーの平均における周波数である。パワーの重心（の周波数）を求めるようなもので，次の式で与えられる：
$$
f_\mathrm{MP}=\frac{\sum_k{f_k P_k}}{\sum_k{P_k}} \tag{4-1}
$$
ここで，$k$は有効な周波数番号である。このセクションでは，各時間ブロックの平均パワー周波数をもとめ，それの時間的変化と相関関係を求める。

□4-1. 式(4-1)の左辺$f_\mathrm{MP}$を`arrf_MP`という変数に格納しよう：  
> 1. (4-1)の右辺の分母は`arrP_eff`の各要素の総和からなる1D-ndarray（教科書p.128参照）。
> 2. (4-1)の右辺の分子は`arrf_eff`と`arrP_eff`のアダマール積（教科書p.107参照）の1D-ndarrayの各要素の総和からなる1D-ndarray。
> 3. (4-1)の右辺は上記1を上記2で（アダマール的に）除した1D-ndarray（教科書p.107参照）。

□4-2. ここでもセクション3-2で定義した中間時刻`tarr_mid`を時間$t$として使うことにする。  

$t$(=`arrt_mid`)と$f_\mathrm{MP}$(=`arrf_MP`)の散布図を描いてみる：

> 直線的に減少しているようだ。

□4-3. セクション4-2のグラフから，$t$(=`arrt_mid`)と$f_\mathrm{MP}$(=`arrf_MP`)にフィットする直線の方程式を求めてみる。  

$t$(=`arrt_mid`)を$x$，$f_\mathrm{MP}$(=`arrP_MP`)を$y$とし，直線$y=a_2x+b_2$の$a_2$に対応する変数を`a2`，$b_2$に対応する変数を`b2`として，`a2`，`b2`を公式2にしたがって求めよう：

□4-4. `a2 * arrt_mid + b2`によって与えれる値の1D-ndarrayを変数`arrf_fit`に格納する： 

□4-5. 上で求めた直線を，セクション4-2の散布図の中に描いてみよう：
> - 図のサイズは`(8, 6)`
> - `arrt_mid`と`arrf_MP`の間の関係は青い点
> - `arrt_mid`と`arrf_fit`の間の関係は赤の曲線
> - x軸ラベルは`'t[sec]'`
> - y軸ラベルは`'mean power frequency'`
> - 凡例は`['data', 'fitting line']`
> - グリッド線を引く

> すばらしい！ データ点がほぼ完全にフィッティング直線に沿っている。

□4-6. セクション3-6と同様に，$t$(=`arrt_mid`)と$f_\mathrm{MP}$(=`arrf_MP`)のあいだの相関関係を調べてみよう：

□4-7. セクション4-6の結果を下のマークダウン セルに述べよ。

（ここをダブルクリックして記述）